In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('consulLog').getOrCreate()

In [9]:
data = spark.read.csv('/Users/hamsinisankaran/Downloads/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Logistic_Regression/customer_churn.csv', header=True,inferSchema=True)

In [10]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [11]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [12]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [13]:
from pyspark.ml.feature import VectorAssembler

In [14]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [15]:
output = assembler.transform(data)

In [16]:
final_data = output.select('features','churn')

In [17]:
train_churn, test_churn = final_data.randomSplit([0.7,0.3])

In [18]:
from pyspark.ml.classification import LogisticRegression

In [19]:
lr_churn =LogisticRegression(labelCol='churn')

In [20]:
fitted_churn_model = lr_churn.fit(train_churn)

In [21]:
training_sum = fitted_churn_model.summary

In [26]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                628|                628|
|   mean|0.17038216560509553| 0.1321656050955414|
| stddev|0.37626789016320605|0.33894068672576644|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [28]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [30]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [31]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|    0|[4.39257553671541...|[0.98778228705498...|       0.0|
|[28.0,11204.23,0....|    0|[1.72998828235306...|[0.84941092143254...|       0.0|
|[29.0,9617.59,0.0...|    0|[4.41020574614153...|[0.98799323661952...|       0.0|
|[29.0,13255.05,1....|    0|[4.01933219800574...|[0.98235208606279...|       0.0|
|[30.0,6744.87,0.0...|    0|[3.39107270479929...|[0.96742436736230...|       0.0|
|[30.0,7960.64,1.0...|    1|[2.72480144043911...|[0.93847435442668...|       0.0|
|[30.0,8677.28,1.0...|    0|[4.09333131664732...|[0.98359021109001...|       0.0|
|[30.0,11575.37,1....|    1|[3.84251764376421...|[0.97901045045116...|       0.0|
|[30.0,12788.37,0....|    0|[2.47871944107237...|[0.92263644302327...|       0.0|
|[31.0,5304.6,0.

In [32]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='churn')

In [34]:
AUC = churn_eval.evaluate(pred_and_labels.predictions)

In [35]:
AUC

0.7979587691682746

In [36]:
#predict on a new data

In [37]:
final_lr_model = lr_churn.fit(final_data)

In [38]:
new_customer = spark.read.csv('/Users/hamsinisankaran/Downloads/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Logistic_Regression/new_customers.csv',inferSchema=True, header=True)

In [39]:
new_customer.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [41]:
test_new_customers = assembler.transform(new_customer)

In [42]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [45]:
final_results = final_lr_model.transform(test_new_customers)

In [46]:
final_results.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|            features|       rawPrediction|         probability|prediction|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|[37.0,9935.53,1.0...|[2.22168705251440...|[0.90218018099704...|       0.0|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|[23.0,7526.94,1.0...|[-6.2207530595020...|[0.00198380445828...|       

In [47]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



In [48]:
test_new_customers.describe().show()

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|summary|        Names|               Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|         Company|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|  count|            6|                 6|                6|                 6|                6|                 6|                   6|               6|
|   mean|         null|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334|                null|            null|
| stddev|         null| 15.71517313511584|4346.008232825459| 0.408248290463863|3.708737880555414|3.3862466931200785|                null|            null|
|    min|Andrew Mccall|              22.0|            100.0|          